In [1]:
using Distributions

In [2]:
type Individual
    m::Int64
    p::Int64
    v_e::Float64
    mu::Float64
    mu_b::Float64
    mu_sigma::Float64
    y::Array{Float64, 1}
    b::Array{Int64, 2}
    x::Array{Float64, 1}
    z::Array{Float64, 1}
    
    Individual(m, p, v_e, mu, mu_b, mu_sigma) = new(2*m, p, v_e, mu, mu_b, mu_sigma)
end  

type Population
    n::Int64
    base_ind::Individual
    generation::Int64
    fitness::Array{Float64, 1}
    theta::Array{Float64, 1}
    omega::Array{Float64, 2}
    pop::Array{Individual, 1}
    Population(n, base_ind, generation) = new(n, base_ind, 0)
end

In [186]:
function Population(n::Int, ind::Individual, pop::Array{Individual, 1})
    new_pop = Population(n, ind)
    new_pop.generation = 0
    new_pop.pop = pop
    new_pop
end

function RandomInd!(ind::Individual, y_mean::Float64 = 0.0, g_sigma::Float64 = ind.mu_sigma)
    ind.y = rand(Normal(y_mean, g_sigma), ind.m)
    ind.b = rand(Bernoulli(), ind.p, ind.m)
    ind.x = ind.b * ind.y
    ind.z = ind.x + rand(Normal(0, ind.v_e), ind.p)
end

function RandomInd(ind::Individual, y_mean::Float64 = 0.0, g_sigma::Float64 = ind.mu_sigma)
    new_ind = Individual(ind.m/2, ind.p, ind.v_e, ind.mu, ind.mu_b, ind.mu_sigma)
    RandomInd!(new_ind, y_mean, g_sigma)
    new_ind
end

function RandomPop!(pop::Population, y_mean::Float64 = 0.0, g_sigma::Float64 = ind.mu_sigma)
    pop.pop = Array(Individual, pop.n)
    pop.generation = 0
    for i = 1:pop.n
        pop.pop[i] = RandomInd(pop.base_ind, y_mean, g_sigma)
    end
end


import Base.getindex

function getindex(pop::Population, i::Integer)
    getindex(pop.pop, i)
end

function getindex(pop::Population, s::UnitRange)
    Population(length(s), pop.base_ind, getindex(pop.pop, s))
end

function append!(pop::Population, ind::Individual)
    pop.pop = [pop.pop; ind]
    pop.n = length(pop.pop)
end

function append!(pop1::Population, pop2::Population)
    # COMPARE BASE IND!
    pop1.pop = [pop1.pop; pop2.pop]
    pop.n = length(pop1.pop)
end

function join(pop1::Population, pop2::Population)
    # COMPARE BASE IND!
    new_pop = Population(pop1.n + pop2.n, pop1.base_ind)
    new_pop.pop = [pop1.pop; pop2.pop]
    new_pop
end

function copy!(source::Population, sink::Population)
    sink.n = source.n
    sink.base_ind = source.base_ind
    sink.generation = source.generation
    sink.fitness = source.fitness
    sink.theta = source.theta
    sink.omega = source.omega
    sink.pop = source.pop
end

function copy(source::Population)
    new_pop = Population(source.n, source.base_ind)
    new_pop.generation = source.generation
    new_pop.fitness = fitness(source)
    new_pop.theta = source.theta
    new_pop.omega = source.omega
    new_pop.pop = source.pop
    new_pop
end
    
function mutation!(ind::Individual)
    mutation_y = rand(Binomial(ind.m, ind.mu))
    if(mutation_y > 0)
        for k = 1:mutation_y
            i = rand(DiscreteUniform(1, ind.m))
            ind.y[i] += rand(Normal(0, ind.mu_sigma))
        end
    end
    mutation_b = rand(Binomial(ind.m * ind.p, ind.mu_b))
    if(mutation_b > 0)
        for k = 1:mutation_b
            i = rand(DiscreteUniform(1, ind.p))
            j = rand(DiscreteUniform(1, ind.m))
            ind.b[i,j] = ind.b[i, j] == 1 ? 0 : 1
        end
    end
end

function mutation!(pop::Population)
    for k = 1:pop.n
        mutation!(pop.pop[k])
    end
end

function fitness(ind::Individual, theta::Array{Float64, 1}, omega::Array{Float64, 2})
    ind_fitness = pdf(MvNormal(theta, omega), ind.z)
    isnan(ind_fitness) ? 0 : ind_fitness
end

function fitness(pop::Population, theta::Array{Float64, 1}, omega::Array{Float64, 2})
    fitness_array = [fitness(pop.pop[k], theta, omega) for k = 1:pop.n]
    w_max = findmax(fitness_array)[1]
    fitness_array/(w_max == 0.0 ? 1 : w_max)
end

function fitness(pop::Population)
    fitness_array = [fitness(pop.pop[k], pop.theta, pop.omega) for k = 1:pop.n]
    w_max = findmax(fitness_array)[1]
    fitness_array/(w_max == 0.0 ? 1 : w_max)
end

function fitness!(pop::Population, theta::Array{Float64, 1}, omega::Array{Float64, 2})
    pop.fitness = fitness(pop, theta, omega)
end

function fitness!(pop::Population)
    pop.fitness = fitness(pop)
end

function cross(ind_1::Individual, ind_2::Individual, new_ind::Individual)
    alele_1 = rand(DiscreteUniform(0, 1), Int(ind_1.m/2))
    alele_2 = rand(DiscreteUniform(0, 1), Int(ind_2.m/2))
    for locus = 1:ind_1.m/2
        new_ind.y[   2 * locus - 1] = ind_1.y[   (2 * locus - 1) + alele_1[locus]]
        new_ind.y[   2 * locus]     = ind_2.y[   (2 * locus - 1) + alele_2[locus]]
        new_ind.b[:, 2 * locus - 1] = ind_1.b[:, (2 * locus - 1) + alele_1[locus]]
        new_ind.b[:, 2 * locus]     = ind_2.b[:, (2 * locus - 1) + alele_2[locus]]
    end
    new_ind.x = new_ind.b * new_ind.y
    new_ind.z = new_ind.x + rand(Normal(0, new_ind.v_e), new_ind.p)
end

function cross(ind_1::Individual, ind_2::Individual)
    new_ind = Individual(ind_1.m/2, ind_1.p, ind_1.v_e, ind_1.mu, ind_1.mu_b, ind_1.mu_sigma)
    new_ind.y = zeros(Float64, new_ind.m)
    new_ind.b = zeros(Float64, new_ind.p, new_ind.m)
    cross(ind_1, ind_2, new_ind)
    new_ind
end   

function choose_mates(pop::Population)
    matings = rand(Multinomial(pop.n, pop.fitness/sum(pop.fitness)), 1)
    mates = zeros(Float64, pop.n)
    l = 1
    for k = 1:pop.n
        if(matings[k] > 0)
            for(i = 1:matings[k])
                mates[l] = k
                l += 1
            end
        end
    end
    round(Int64, shuffle!(mates))
end

function next_generation!(pop::Population)
    holder_pop = copy(pop)
    next_generation!(pop, holder_pop)
end

function next_generation!(pop::Population, holder_pop::Population)
    fitness!(pop)
    mutation!(pop)
    sires = choose_mates(pop)
    dames = choose_mates(pop)
    for i in 1:pop.n
        holder_pop.pop[i] = cross(pop[sires[i]], pop[dames[i]])
    end
    holder_pop.generation += 1
    copy!(holder_pop, pop)
end 

function momments(pop::Population)
    ys = convert(Array{Float64, 2}, reshape([ind.y[i] for ind in pop.pop, i in 1:pop.base_ind.m], pop.n, pop.base_ind.m))
    xs = convert(Array{Float64, 2}, reshape([ind.x[i] for ind in pop.pop, i in 1:pop.base_ind.p], pop.n, pop.base_ind.p))
    zs = convert(Array{Float64, 2}, reshape([ind.z[i] for ind in pop.pop, i in 1:pop.base_ind.p], pop.n, pop.base_ind.p))
    mean_b = zeros(Float64, pop.base_ind.p, pop.base_ind.m)
    for i in 1:pop.n
        mean_b += pop[i].b
    end
    mean_y = squeeze(mean(ys, 1), 1)
    mean_x = squeeze(mean(xs, 1), 1)
    mean_z = squeeze(mean(zs, 1), 1)
    mean_b /= pop.n

    P = cov(zs)
    G = cov(xs)
    h2 = diag(G)./diag(P)
    corrG = cor(xs)
    corrP = cor(zs)

    Dict([("mean_y", mean_y), 
          ("mean_b", mean_b), 
          ("mean_x", mean_x),
          ("mean_z", mean_z), 
          ("P", P), 
          ("G", G), 
          ("h2", h2), 
          ("corrG", corrG),
          ("corrP", corrP)])
end

next_generation! (generic function with 2 methods)

In [278]:
ind = Individual(10, 4, 0.8, 0.005, 0.001, 0.2)
pop = Population(20, ind)
RandomPop!(pop)
pop.theta = zeros(Float64, ind.p)
pop.omega = diagm(ones(Float64, ind.p))
fitness!(pop)

20-element Array{Float64,1}:
 0.125744 
 0.357302 
 0.259888 
 1.0      
 0.0539199
 0.781274 
 0.372816 
 0.519454 
 0.334699 
 0.820486 
 0.11931  
 0.583855 
 0.0320036
 0.013769 
 0.0106091
 0.428074 
 0.251453 
 0.105598 
 0.333158 
 0.238085 

In [213]:
next_generation!(pop)

20-element Array{Individual,1}:
 Individual(20,4,0.2,0.005,0.001,0.8,[-0.937998703203715,0.197074632581463,1.1037459807138887,1.1037459807138887,0.325096288793123,-0.46705606619441387,-1.0698996453342355,-1.8529666011309414,0.4808578986723644,0.1389352776596249,0.17353540109391385,0.17353540109391385,-0.3224992218596361,0.2850410067377932,0.4422507528088008,-0.17772897702053425,-0.7472660412763923,-0.7472660412763923,-0.8427130818394389,-0.8427130818394389],4x20 Array{Int64,2}:
 1  0  1  1  1  1  0  1  1  1  1  1  1  1  0  1  0  0  1  1
 0  0  1  1  0  1  1  1  1  0  1  1  0  1  1  0  0  0  0  0
 1  0  0  0  1  0  0  0  0  1  0  0  1  1  1  1  0  0  0  0
 0  0  0  0  1  0  1  0  0  1  0  0  1  0  0  0  0  0  1  1,[-1.659182497609608,0.3727901091749728,-0.2469035760845435,-2.613793464420002],[-1.632913045914558,0.871743822805626,-0.5483095839582044,-2.6786888628857786])              
 Individual(20,4,0.2,0.005,0.001,0.8,[-0.5276295588006542,-0.32484125468796327,0.697482538055972,0.65734

momments (generic function with 1 method)

In [288]:
m = momments(pop)

Dict{ASCIIString,Array{Float64,N}} with 9 entries:
  "corrP"  => 4x4 Array{Float64,2}:…
  "P"      => 4x4 Array{Float64,2}:…
  "h2"     => [0.11392050995602694,0.4275543298418692,0.32932978091883525,0.559…
  "mean_y" => [0.0797513996895052,-0.03429354584339148,0.06815179316763136,-0.0…
  "mean_b" => 4x20 Array{Float64,2}:…
  "mean_x" => [-0.04508749157658931,-0.07676091771445141,-0.10296992873144371,-…
  "corrG"  => 4x4 Array{Float64,2}:…
  "G"      => 4x4 Array{Float64,2}:…
  "mean_z" => [-0.35956921799498276,-0.03485016780124044,-0.3425900590490532,-0…

4x4 Array{Float64,2}:
 0.176011  0.127326  0.106459  0.091707
 0.127326  0.307164  0.127339  0.209413
 0.106459  0.127339  0.263826  0.246303
 0.091707  0.209413  0.246303  0.518727